In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics

from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklego.metrics import p_percent_score
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report

import sklego
from fairlearn.metrics import MetricFrame
from sklearn.metrics import accuracy_score

In [2]:
##READ DATA 
data_df = pd.read_csv('dataset_ready2.csv') #320.915 total rows/companies 
data_df

,Unnamed: 0,ORG_org_uuid,company_name,region,city,status,multiple_degrees_sum,multiple_degrees_average,is_completed_sum,is_completed_avg,...,female_founders,unknown_founders,total_num_founders,category_list1,category_list2,category_list3,category_group_list1,category_group_list2,category_group_list3,mostly_male_founders
0,0,00001b2d-a4f7-55d5-d69a-17acbac9c17b,STD Risk Calculator,0.000000,0.000000,operating,0.0,0.0,0.0,0.0,...,0,1,1,66,0,0,6,39,0,0
1,1,00002470-bff7-6226-5800-0ca1b3787b6f,Codementor,0.000000,0.748085,operating,2.0,1.0,2.0,1.0,...,0,0,4,189,206,24821,15,40,0,1
2,2,00003a2c-ad4d-ff61-997b-7b22b2320895,Smart MultiMedia,0.732143,0.733041,operating,0.0,0.0,0.0,0.0,...,0,1,1,131,169,5428,13,13,4774,0
3,3,000095de-8e2b-82f1-32a7-c222ba3d5682,Ultraprise Loan Technologies,0.681548,0.732135,operating,0.0,0.0,0.0,0.0,...,0,1,1,51,196,0,8,0,0,0
4,4,0000d497-c93a-eea3-eeb0-a943dfb4f71e,AutoOffer,0.610863,0.687143,operating,0.0,0.0,0.0,0.0,...,0,0,1,49,353,24563,18,25,6667,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320910,320910,ffff25bc-9902-33e8-062e-6d84841d8cf9,Vivoxid,0.719494,0.737359,closed,0.0,0.0,0.0,0.0,...,0,1,1,314,0,0,23,0,0,0
320911,320911,ffff3209-5481-022e-cdbf-c38a0a762dc4,BJJ Jiu-Jitsu,0.000000,0.000000,operating,0.0,0.0,0.0,0.0,...,0,1,1,364,0,0,25,0,0,0
320912,320912,ffff8343-97a6-3667-67af-ed3789fa9e14,Wantist,0.596726,0.683939,operating,0.0,0.0,0.0,0.0,...,1,0,2,205,291,23667,8,17,0,0
320913,320913,ffffabce-6d4a-b3d1-13c0-4e90cedf5270,ERNIT,0.519345,0.672656,operating,0.0,0.0,0.0,0.0,...,0,0,6,235,256,18366,4,17,6281,1


In [3]:
#DROP ALL "operating"
data_df = data_df.drop(data_df[data_df.status == 'operating'].index) #leaves 61,979 companies

In [4]:
#DROP all that are with unknown gender of founders
data_df = data_df.drop(data_df[data_df.unknown_founders != 0].index)

In [5]:
#set org uuid as the index 
data_df = data_df.set_index('ORG_org_uuid')

In [6]:
#create the y variable 
y_variable = data_df.status

#Drop unnecessary columns 
data_x = data_df.drop(['company_name', 'Unnamed: 0', 'status'], axis = 1)

## CREATE A BINARY OUTCOME VARIABLE
#y_variable_bin = y_variable.replace('operating', 1, regex = True)
y_variable_bin = y_variable.replace('ipo', 1, regex = True)
y_variable_bin = y_variable_bin.replace('acquired', 1, regex = True)
y_variable_bin = y_variable_bin.replace('closed', 0, regex = True)

X = data_x
y = y_variable_bin 

In [7]:
from collections import Counter
counter = Counter(y)
print(counter) #41.903 in the positive class, 20.076 in negative 

Counter({1: 13726, 0: 4767})


In [8]:
X.mostly_male_founders.value_counts()

1    17619
0      874
Name: mostly_male_founders, dtype: int64

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


In [10]:
X_train = X_train.drop(labels=['female_founders', 'male_founders', 'mostly_male_founders'], axis=1)
X_test_drop = X_test.drop(labels=['female_founders', 'male_founders', 'mostly_male_founders'], axis=1)



In [ ]:

print("Before OverSampling, counts of label '1': {}".format(sum(y_train == 1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train == 0)))
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state = 2)
X_train, y_train = sm.fit_resample(X_train, y_train.ravel())
  
print('After OverSampling, the shape of train_X: {}'.format(X_train.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(X_train.shape))
  
print("After OverSampling, counts of label '1': {}".format(sum(y_train == 1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train == 0)))

## Model training - with gender variables 

In [11]:
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import StandardScaler
##SVD needs some work on no of components

_sc = MinMaxScaler()
_svd = TruncatedSVD(n_components=10)
#_pca = PCA(n_components = components)
_model = LogisticRegression()
log_regress_model = Pipeline([
    ('std_scaler', _sc),
    ('svd', _svd),
    ('regressor', _model)
])

In [12]:

###TRAIN A MODEL WITH ALL VARIABLES 
logreg_all = LogisticRegression()
#logreg_all.fit(X_train_res, y_train_res)

log_regress_model.fit(X_train, y_train)



Pipeline(steps=[('std_scaler', MinMaxScaler()),
                ('svd', TruncatedSVD(n_components=10)),
                ('regressor', LogisticRegression())])

In [13]:
# Predicting on the test data
y_pred2 = log_regress_model.predict(X_test_drop)

#Calculating and printing the f1 score 
f1_test = f1_score(y_test, y_pred2)

#Ploting the confusion matrix
confusion_matrix(y_test, y_pred2)

print(classification_report(y_test, y_pred2))

gm1 = MetricFrame(metrics=accuracy_score, y_true=y_test, y_pred=y_pred2, sensitive_features = X_test.mostly_male_founders)
print("Accuracy overall \n", gm1.overall, "\n")
print("Accuracy by group",
      "\n 0 mostly female, 1 mostly male \n",
      "\n", gm1.by_group 
      )


              precision    recall  f1-score   support

           0       0.71      0.34      0.46      1163
           1       0.81      0.95      0.88      3461

    accuracy                           0.80      4624
   macro avg       0.76      0.65      0.67      4624
weighted avg       0.79      0.80      0.77      4624

Accuracy overall 
 0.7988754325259516 

Accuracy by group 
 0 mostly female, 1 mostly male 
 
 mostly_male_founders
0    0.719665
1    0.803193
Name: accuracy_score, dtype: object
